In [304]:
#!pip install torch
#!pip install torchvision

In [305]:
import torch.nn as nn
import torchvision

In [306]:
import pandas as pd
import numpy as np

In [307]:
df = pd.read_csv("preprocessed_data.csv")
df.describe()

,total_load,latitude,longitude,temperature,hour,day_of_week_num,day_of_week_sin,day_of_week_cos,hour_sin,hour_cos,...,next_load_14,next_load_15,next_load_16,next_load_17,next_load_18,next_load_19,next_load_20,next_load_21,next_load_22,next_load_23
count,43848.000000,43848.0,4.384800e+04,43848.000000,43848.000000,43848.000000,4.384800e+04,4.384800e+04,4.384800e+04,4.384800e+04,...,43848.000000,43848.000000,43848.000000,43848.000000,43848.000000,43848.000000,43848.000000,43848.000000,43848.000000,43848.000000
mean,19201.408502,52.0,1.955556e+01,9.957783,11.500000,3.000000,-2.139018e-17,-1.920254e-17,-1.361193e-17,-5.534910e-17,...,19201.688538,19201.663747,19201.636996,19201.583379,19201.522692,19201.460203,19201.396255,19201.338533,19201.297140,19201.272259
std,3246.750377,0.0,1.670150e-11,8.275852,6.922265,2.000023,7.071148e-01,7.071148e-01,7.071148e-01,7.071148e-01,...,3246.286491,3246.316779,3246.349109,3246.404176,3246.463605,3246.524035,3246.585244,3246.642919,3246.688841,3246.719218
min,10769.000000,52.0,1.955556e+01,-17.130000,0.000000,0.000000,-9.749279e-01,-9.009689e-01,-1.000000e+00,-1.000000e+00,...,10769.000000,10769.000000,10769.000000,10769.000000,10769.000000,10769.000000,10769.000000,10769.000000,10769.000000,10769.000000
25%,16555.000000,52.0,1.955556e+01,3.300000,5.750000,1.000000,-7.818315e-01,-9.009689e-01,-7.071068e-01,-7.071068e-01,...,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16554.812500,16554.187500,16554.000000,16554.000000,16554.000000
50%,19252.000000,52.0,1.955556e+01,9.324444,11.500000,3.000000,0.000000e+00,-2.225209e-01,6.123234e-17,-6.123234e-17,...,19252.000000,19252.000000,19252.000000,19252.000000,19252.000000,19252.000000,19252.000000,19252.000000,19252.000000,19252.000000
75%,21664.000000,52.0,1.955556e+01,16.307778,17.250000,5.000000,7.818315e-01,6.234898e-01,7.071068e-01,7.071068e-01,...,21664.000000,21664.000000,21664.000000,21664.000000,21664.000000,21664.000000,21664.000000,21664.000000,21664.000000,21664.000000
max,28304.000000,52.0,1.955556e+01,32.534444,23.000000,6.000000,9.749279e-01,1.000000e+00,1.000000e+00,1.000000e+00,...,28304.000000,28304.000000,28304.000000,28304.000000,28304.000000,28304.000000,28304.000000,28304.000000,28304.000000,28304.000000


In [308]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43848 entries, 0 to 43847
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             43848 non-null  object 
 1   total_load       43848 non-null  float64
 2   latitude         43848 non-null  float64
 3   longitude        43848 non-null  float64
 4   temperature      43848 non-null  float64
 5   date             43848 non-null  object 
 6   hour             43848 non-null  int64  
 7   day_of_week      43848 non-null  object 
 8   day_of_week_num  43848 non-null  int64  
 9   day_of_week_sin  43848 non-null  float64
 10  day_of_week_cos  43848 non-null  float64
 11  hour_sin         43848 non-null  float64
 12  hour_cos         43848 non-null  float64
 13  day_of_year      43848 non-null  int64  
 14  days_in_year     43848 non-null  int64  
 15  day_of_year_sin  43848 non-null  float64
 16  day_of_year_cos  43848 non-null  float64
 17  load-1      

In [309]:
df['date'] = pd.to_datetime(df['time'])
df['date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 43848 entries, 0 to 43847
Series name: date
Non-Null Count  Dtype         
--------------  -----         
43848 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 342.7 KB


In [310]:
from torch.utils.data import Dataset, DataLoader

In [311]:
from torch.utils.data import Dataset

class PowerLoadDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        X, y = self.x_y(self.df.iloc[idx])
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

    @staticmethod
    def x_y(row):
        feature_cols = ['load-1', 'load-2', 'load-3', 'load-22', 'load-23', 'load-24', 'load-25', 'load-26', 'mean_t_3',
                        'mean_t_5', 'day_of_week_sin', 'day_of_week_cos', 'hour_sin', 'hour_cos', 'day_of_year_sin', 'day_of_year_cos']
        target_cols = ['total_load'] + [f'next_load_{i}' for i in range(1, 24)]
        X = row[feature_cols].values.astype('float32')
        y = row[target_cols].values.astype('float32')
        return X, y
    

In [312]:
train_df = df[df['date'].dt.year < 2024]
test_df = df[df['date'].dt.year == 2024]

In [313]:
from sklearn.preprocessing import MinMaxScaler

In [314]:
#Normalization
from sklearn.preprocessing import MinMaxScaler

temp_cols = ['mean_t_3', 'mean_t_5']
other_cols = ['load-1', 'load-2', 'load-3', 'load-22', 'load-23', 'load-24', 'load-25', 'load-26',
              'day_of_week_sin', 'day_of_week_cos', 'hour_sin', 'hour_cos', 'day_of_year_sin', 'day_of_year_cos']

scaler_temp = MinMaxScaler(feature_range=(-1, 1)).fit(train_df[temp_cols])
train_df[temp_cols] = scaler_temp.transform(train_df[temp_cols])
test_df[temp_cols] = scaler_temp.transform(test_df[temp_cols])

for col in other_cols:
    scaler = MinMaxScaler()
    scaler.fit(train_df[[col]])
    train_df[col] = scaler.transform(train_df[[col]])
    test_df[col] = scaler.transform(test_df[[col]])
    
target_cols = ['total_load'] + [f'next_load_{i}' for i in range(1, 24)]
scaler_y = MinMaxScaler()
scaler_y.fit(train_df[target_cols])
train_df[target_cols] = scaler_y.transform(train_df[target_cols])
test_df[target_cols] = scaler_y.transform(test_df[target_cols])


C:\Users\DELL\AppData\Local\Temp\ipykernel_10028\2206988589.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[temp_cols] = scaler_temp.transform(train_df[temp_cols])
C:\Users\DELL\AppData\Local\Temp\ipykernel_10028\2206988589.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[temp_cols] = scaler_temp.transform(test_df[temp_cols])
C:\Users\DELL\AppData\Local\Temp\ipykernel_10028\2206988589.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [315]:
train_dataset = PowerLoadDataset(train_df)
test_dataset = PowerLoadDataset(test_df)

In [316]:
features_train, labels_train = train_dataset[0]
features_test, labels_test = test_dataset[0]
print(f"TRAIN: Shape of features - {features_train.shape}, Shape of labels - {labels_train.shape}")
print(f"TEST: Shape of features - {features_test.shape}, Shape of labels - {labels_test.shape}")

TRAIN: Shape of features - torch.Size([16]), Shape of labels - torch.Size([24])
TEST: Shape of features - torch.Size([16]), Shape of labels - torch.Size([24])


In [317]:
print(f"Number of samples in test dataset: {len(train_dataset)}")
print(f"Number of samples in train dataset: {len(test_dataset)}")

combined_samples_number = len(train_dataset) + len(test_dataset)
print(df.shape[0])
print(combined_samples_number == df.shape[0])

Number of samples in test dataset: 35064
Number of samples in train dataset: 8784
43848
True


In [318]:
dataloader_train = DataLoader(train_dataset, batch_size=32, shuffle=False)
dataloader_test = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [319]:
features, labels = next(iter(dataloader_train))
print(f"Features shape: {features.shape}, labels shape: {labels.shape} ")

Features shape: torch.Size([32, 16]), labels shape: torch.Size([32, 24]) 


In [320]:
class SimplePerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16, 25)
        self.fc2 = nn.Linear(25, 24)
    
    def forward(self, x):
        x = nn.functional.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [321]:
Perceptron = SimplePerceptron()

### Training Loop

In [322]:
import torch.optim as optim

In [323]:
optimizer = optim.Adam(Perceptron.parameters(), lr=0.0005)

In [324]:
criterion = nn.MSELoss()

Our metric will be mape

In [325]:
import torch

In [326]:
def mape(y_true, y_pred):
    mask = y_true != 0
    return (torch.abs((y_true[mask] - y_pred[mask]) / (y_true[mask] + 1e-8))).mean() * 100

In [327]:
import numpy as np

for epoch in range(20):
    running_loss = 0.0
    running_mape = 0.0
    for features, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = Perceptron(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        outputs_np = outputs.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()
        outputs_orig = scaler_y.inverse_transform(outputs_np)
        labels_orig = scaler_y.inverse_transform(labels_np)
        
        batch_mape = np.mean(np.abs((labels_orig - outputs_orig) / (labels_orig + 1e-8))) * 100
        running_mape += batch_mape
        
    avg_loss = running_loss / len(dataloader_train)
    avg_mape = running_mape / len(dataloader_train)
    print(f"Epoch {epoch+1}: Loss={avg_loss:.4f}, MAPE={avg_mape:.2f}%")

Epoch 1: Loss=0.0447, MAPE=14.85%
Epoch 2: Loss=0.0221, MAPE=11.32%
Epoch 3: Loss=0.0157, MAPE=9.06%
Epoch 4: Loss=0.0133, MAPE=8.03%
Epoch 5: Loss=0.0124, MAPE=7.66%
Epoch 6: Loss=0.0118, MAPE=7.47%
Epoch 7: Loss=0.0115, MAPE=7.33%
Epoch 8: Loss=0.0111, MAPE=7.21%
Epoch 9: Loss=0.0108, MAPE=7.10%
Epoch 10: Loss=0.0105, MAPE=6.98%
Epoch 11: Loss=0.0102, MAPE=6.86%
Epoch 12: Loss=0.0099, MAPE=6.74%
Epoch 13: Loss=0.0096, MAPE=6.61%
Epoch 14: Loss=0.0093, MAPE=6.48%
Epoch 15: Loss=0.0091, MAPE=6.37%
Epoch 16: Loss=0.0089, MAPE=6.27%
Epoch 17: Loss=0.0087, MAPE=6.18%
Epoch 18: Loss=0.0085, MAPE=6.11%
Epoch 19: Loss=0.0084, MAPE=6.05%
Epoch 20: Loss=0.0083, MAPE=5.99%


### Model Evaluation


In [329]:
Perceptron.eval()

total_loss = 0.0
total_mape = 0.0
n_batches = 0

with torch.no_grad():
    for features, labels in dataloader_test:
        outputs = Perceptron(features)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        
        outputs_np = outputs.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()
        outputs_orig = scaler_y.inverse_transform(outputs_np)
        labels_orig = scaler_y.inverse_transform(labels_np)
        
        batch_mape = np.mean(np.abs((labels_orig - outputs_orig) / (labels_orig + 1e-8))) * 100
        total_mape += batch_mape
        n_batches += 1

avg_loss = total_loss / n_batches
avg_mape = total_mape / n_batches
print(f"Test MSE: {avg_loss:.4f}, Test MAPE: {avg_mape:.2f}%")

Test MSE: 0.0109, Test MAPE: 7.34%
